In [2]:
import pandas as pd
import numpy as np 
import os
np.set_printoptions(threshold=np.inf)

In [3]:
cwd = os.getcwd()
print(cwd)

/Users/nathan/Documents/Stats/Bosch Kaggle/bosch


In [6]:
# Read the first few rows
datapath = cwd + '/../'
#categorical = pd.read_csv(datapath + 'train_categorical.csv', nrows=100)
dates = pd.read_csv(datapath + 'train_date.csv', nrows=100)
dates_test = pd.read_csv(datapath + 'test_date.csv', nrows=100)
#numerics = pd.read_csv(datapath + 'train_numeric.csv', nrows=100)

In [7]:
# Make a 
# Make a date_ids list of lists of ids operated on for each date
# Make a date_response file with a count of success_ids and fail_ids 
# processed on each date

all_dates = list(dates)
print(len(all_dates), all_dates[0:2])
all_dates = all_dates[1:]
print(len(all_dates), all_dates[0:2])

1157 ['Id', 'L0_S0_D1']
1156 ['L0_S0_D1', 'L0_S0_D3']


In [8]:
all_dates_test = list(dates_test)
all_dates_test = all_dates_test[1:]
print(set(all_dates)^set(all_dates_test))


set()


In [10]:
print(len(all_dates), len(all_dates_test))

1156 1156


In [106]:
date_iter_csv = pd.read_csv(datapath + 'train_date.csv',
                            iterator = True,
                            chunksize = 10000,
                            index_col = 0)
date_ids = {}
chunk_num = 0
for chunk in date_iter_csv:

    for j in chunk.itertuples():
        # this is a single row, which is a single id
        # I will add this ID to every date's list of ids for which
        # this ID has an entry
        for c in range(0, len(j)-1):
            if pd.isnull(j[c+1]) == False:
                # j[0] is the id
                temp = date_ids.get(all_dates[c], [])
                temp.append(j[0])
                date_ids[all_dates[c]] = temp
            print(chunk_num, "/119 with dates: ", len(date_ids))
            chunk_num += 1

0  with dates:  1145
1  with dates:  1145
2  with dates:  1149
3  with dates:  1149
4  with dates:  1150
5  with dates:  1150
6  with dates:  1150
7  with dates:  1150
8  with dates:  1150
9  with dates:  1150
10  with dates:  1150
11  with dates:  1150
12  with dates:  1150
13  with dates:  1150
14  with dates:  1150
15  with dates:  1150
16  with dates:  1154
17  with dates:  1154
18  with dates:  1154
19  with dates:  1154
20  with dates:  1154
21  with dates:  1154
22  with dates:  1154
23  with dates:  1154
24  with dates:  1154
25  with dates:  1154
26  with dates:  1154
27  with dates:  1154
28  with dates:  1154
29  with dates:  1154
30  with dates:  1154
31  with dates:  1154
32  with dates:  1154
33  with dates:  1154
34  with dates:  1155
35  with dates:  1155
36  with dates:  1155
37  with dates:  1155
38  with dates:  1155
39  with dates:  1155
40  with dates:  1155
41  with dates:  1155
42  with dates:  1155
43  with dates:  1155
44  with dates:  1155
45  with dates:  115

In [12]:
test_date_iter_csv = pd.read_csv(datapath + 'test_date.csv',
                            iterator = True,
                            chunksize = 10000,
                            index_col = 0)
test_date_ids = {}
chunk_num = 0
for chunk in test_date_iter_csv:

    for j in chunk.itertuples():
        # this is a single row, which is a single id
        # I will add this ID to every date's list of ids for which
        # this ID has an entry
        for c in range(0, len(j)-1):
            if pd.isnull(j[c+1]) == False:
                # j[0] is the id
                temp = test_date_ids.get(all_dates[c], [])
                temp.append(j[0])
                test_date_ids[all_dates[c]] = temp
    print(chunk_num, "/119 with dates: ", len(test_date_ids))
    chunk_num += 1

0 /119 with dates:  1145
1 /119 with dates:  1145
2 /119 with dates:  1145
3 /119 with dates:  1145
4 /119 with dates:  1154
5 /119 with dates:  1154
6 /119 with dates:  1154
7 /119 with dates:  1154
8 /119 with dates:  1154
9 /119 with dates:  1154
10 /119 with dates:  1154
11 /119 with dates:  1154
12 /119 with dates:  1154
13 /119 with dates:  1154
14 /119 with dates:  1154
15 /119 with dates:  1154
16 /119 with dates:  1154
17 /119 with dates:  1154
18 /119 with dates:  1154
19 /119 with dates:  1154
20 /119 with dates:  1154
21 /119 with dates:  1154
22 /119 with dates:  1154
23 /119 with dates:  1154
24 /119 with dates:  1154
25 /119 with dates:  1154
26 /119 with dates:  1154
27 /119 with dates:  1154
28 /119 with dates:  1154
29 /119 with dates:  1154
30 /119 with dates:  1154
31 /119 with dates:  1154
32 /119 with dates:  1154
33 /119 with dates:  1154
34 /119 with dates:  1154
35 /119 with dates:  1154
36 /119 with dates:  1154
37 /119 with dates:  1154
38 /119 with dates:  1

In [13]:
test_date_ids_series = pd.Series(test_date_ids)
print(test_date_ids_series.head())
test_date_ids_series.to_csv('../test_date_ids.csv')

L0_S0_D1     [5, 10, 12, 15, 20, 21, 22, 24, 25, 29, 30, 33...
L0_S0_D11    [5, 10, 12, 15, 20, 21, 22, 24, 25, 29, 30, 33...
L0_S0_D13    [5, 10, 12, 15, 20, 21, 22, 24, 25, 29, 30, 33...
L0_S0_D15    [5, 10, 12, 15, 20, 21, 22, 24, 25, 29, 30, 33...
L0_S0_D17    [5, 10, 12, 15, 20, 21, 22, 24, 25, 29, 30, 33...
dtype: object


In [15]:
print(len(test_date_ids))
print(len(all_dates))
print(set(all_dates) - set(test_date_ids.keys()))

1155
1156
{'L3_S46_D4135'}


In [14]:
print(len(date_ids))
print(len(all_dates))
print(set(all_dates) - set(date_ids.keys()))

NameError: name 'date_ids' is not defined

In [105]:
counter = 0
for i in range(len(all_dates)):
  if all_dates[i] in date_ids:
    if len(date_ids[all_dates[i]]) > 50:
      counter += 1
print(counter)

751


In [108]:
date_ids_series = pd.Series(date_ids)
print(date_ids_series.head())
date_ids_series.to_csv('../date_ids.csv')

L0_S0_D1     [4, 7, 9, 11, 13, 18, 26, 27, 28, 31, 34, 38, ...
L0_S0_D11    [4, 7, 9, 11, 13, 18, 26, 27, 28, 31, 34, 38, ...
L0_S0_D13    [4, 7, 9, 11, 13, 18, 26, 27, 28, 31, 34, 38, ...
L0_S0_D15    [4, 7, 9, 11, 13, 18, 26, 27, 28, 31, 34, 38, ...
L0_S0_D17    [4, 7, 9, 11, 13, 18, 26, 27, 28, 31, 34, 38, ...
dtype: object


In [134]:
# make the successfail result code csv
id_list = []
result_list = []
numeric_iter_csv = pd.read_csv(datapath + 'train_numeric.csv',
                            iterator = True,
                            chunksize = 1000,
                            index_col = 0)
chunk_num = 0
for chunk in numeric_iter_csv:
  chunk_num += 1
  #print(chunk_num)
  #print(chunk.columns)
  id_list.extend(chunk.index.tolist())
  result_list.extend(chunk.Response)

print(len(id_list), len(result_list))
successfail = pd.DataFrame({'Id':id_list, 'Response':result_list})
successfail = successfail.set_index('Id')
successfail.to_csv('../response_code.csv')


1183747 1183747


In [133]:
print(successfail.head())
print(id_list[0:5], result_list[0:5])

   Id  Response
0   4         0
1   6         0
2   7         0
3   9         0
4  11         0
[4, 6, 7, 9, 11] [0, 0, 0, 0, 0]


In [171]:
print(id_list.index(1183750))
print(np.where(successfail == 1183750))
successfail = successfail.set_index('Id')
print(successfail.loc[1183750,])

591432
(array([591432]), array([0]))
Response    0
Name: 1183750, dtype: int64


In [142]:
date_ids_df = pd.DataFrame({'date_id_list':date_ids,
                       'successes':0,
                       'failures':0,
                       'totals':0})
print(list(date_ids_df))
print(date_ids_df.head(1), date_ids_df.shape)

['date_id_list', 'failures', 'successes', 'totals']
                                               date_id_list  failures  \
L0_S0_D1  [4, 7, 9, 11, 13, 18, 26, 27, 28, 31, 34, 38, ...         0   

          successes  totals  
L0_S0_D1          0       0   (1156, 4)


In [187]:
for i in range(100,130):
  try:
    print(len(date_ids_df.iloc[i,0]))
  except:
    print(date_ids_df.iloc[i,0])

81368
81368
81368
81368
81368
80600
80591
80599
80595
80599
80591
80591
80591
80591
80591
80591
80591
80591
80591
80591
80290
80290
80290
80290
80290
80290
80290
80290
80290
80290


In [188]:
successes = np.zeros(len(date_ids_df))
failures = np.zeros(len(date_ids_df))
for rownum in range(date_ids_df.shape[0]):
  for item in date_ids_df.iloc[rownum,0]:
    temp_result = successfail.loc[item, 'Response']
    if temp_result == 0:
      successes[rownum] += 1
    elif temp_result == 1:
      failures[rownum] += 1

1156 1156


In [190]:
print(sum(successes) + sum(failures))

80290.0


In [156]:
print(date_ids_df.head(1))


                                               date_id_list  failures  \
L0_S0_D1  [4, 7, 9, 11, 13, 18, 26, 27, 28, 31, 34, 38, ...         0   

          successes  totals  
L0_S0_D1          0    1156  


In [ ]:
print(list(date_ids_df))
print(date_ids_df.head(1), date_ids_df.shape)

In [ ]:
date_ids_df['failures'] = failures
date_ids_df['successes'] = successes
date_ids_df['totals'] = date_ids_df.failures + date_ids_df.successes

In [ ]:
date_ids_df.to_csv('../date_ids_df.csv')